In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import requests
import os
import regex as re
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.
2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [2]:
def Product_Keywords():
    
    #Code will take input of the product we wish to search and return the String 
    search_keyword=input("Please enter the product you wish to search: ")
    return search_keyword


def Amazon_Product_Search(url):
    
    #Calling the Product_Keywords funtion 
    search_keyword=Product_Keywords()
    
    driver=webdriver.Chrome()
    driver.get(url)
    
    wait=WebDriverWait(driver,10)
    
    #Declaring the required lists
    brand_name = []
    name_of_the_product = []
    product_price = []
    return_exchange = []
    expected_delivery = []
    availability = []
    product_url = []

    
    #Code will locate the search field and send the user input
    search=wait.until(EC.visibility_of_element_located((By.XPATH,'//input[@id="twotabsearchtextbox"]')))
    search.send_keys(search_keyword)
    search.send_keys(Keys.ENTER)
    
    #Iterate through maximum of 3 pages
    for pages in range(3):
        products_elements_url=wait.until(EC.presence_of_all_elements_located((By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')))
    
        #Code to locate all the product urls
        for get_url in products_elements_url:
            product_url.append(get_url.get_attribute('href'))
        
        
        driver.find_element(By.XPATH,'//a[contains(text(),"Next")]').click()
        time.sleep(3)
                                                  
    #Iterate through each product url
    for each_product_url in product_url:
        
        
        driver.get(each_product_url)
        time.sleep(3)
        
        #These try except blocks of Code will extract product title, name, price,return/exchange, availability 
        try:
            product_name = driver.find_element(By.XPATH,'//span[@id="productTitle"]')
            name_of_the_product.append(product_name.get_attribute("innerText"))
        except NoSuchElementException:
            name_of_the_product.append('-')
        
        try:
            brand=driver.find_element(By.XPATH,'//tr[@class="a-spacing-small po-brand"]/td[2]')
            brand_name.append(brand.get_attribute("innerText"))
        except NoSuchElementException:
            brand_name.append('-')
        
        try:
            price=driver.find_element(By.XPATH,'//span[@id="tp_price_block_total_price_ww"]/span[2]/span[2]')
            product_price.append(price.get_attribute("innerText"))
        except NoSuchElementException:
            product_price.append('-')
                
        try:
            return_policy = driver.find_element(By.XPATH,'//span[@class="a-size-small a-color-link a-text-normal" and contains(text(),"days")]')
            return_exchange.append(return_policy.get_attribute("innerText"))
        except:
            return_exchange.append('-')
        
        try:
            is_available=driver.find_element(By.XPATH,'//span[@class="a-size-medium a-color-success"]')
            availability.append(is_available.get_attribute("innerText"))
        except:
            availability.append('-')     
    
    #Creating a data frame
    df_amazon_product_search=pd.DataFrame({'Product Name':name_of_the_product,'Brand Name':brand_name,'Price':product_price,'Return/Exchange':return_exchange,'Availability':availability,'URL':product_url},index=range(1,len(name_of_the_product)+1))
    
    #Converting DataFrame into CSV
    df_amazon_product_search.to_csv('Amazon_product_data.csv')
    
    #Returning DataFrame
    return df_amazon_product_search
    

Amazon_Product_Search('https://www.amazon.in/')

Please enter the product you wish to search: Guitar


,Product Name,Brand Name,Price,Return/Exchange,Availability,URL
1,"Intern INT-38C Right hand Acoustic Guitar Kit,...",Intern,"2,299.",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Intern Int-38c Sunburst Linden Wood Acoustic G...,Intern,"1,799.",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Kadence Frontier guitar with Online Guitar lea...,Kadence,"4,499.",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,"Kadence rosewood Guitar Frontier Series, Elect...",Kadence,"5,499.",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
5,"Juârez Acoustic Guitar Kit, 38 Inch Cutaway, 0...",JUAREZ,"1,949.",7 days Replacement,In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...
185,"BLUEBERRY D38S, Acoustic Guitar Kit Inbuilt Tr...",blueberry,"2,849.",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
186,"BLUEBERRY VlP-405, Truss Rod Acoustic Guitar K...",blueberry,"4,361.",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
187,"blueberry, VIP-300, 30 Inch Linden Wood Acoust...",blueberry,"2,655.",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
188,"Festra R-40, Truss Rod Acoustic Guitar Kit wit...",Festra,"4,074.",7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [3]:
def Google_Image_Search():
    
    driver=webdriver.Chrome()
    wait=WebDriverWait(driver,10)
    
    #Creating a list of all the keywords needs to searched
    keywords = ['Fruits','Cars','Machine Learning','Guitar','Cakes']
    
    #Declaring the required lists
    fruits_image_source = []
    cars_image_source = []
    machine_learning_image_source = []
    guitar_image_source = []
    cakes_image_source = []
    

    driver.get('https://images.google.com/')
    
    #Iterating through each items in the keywords list
    for item in keywords:     
        
        #Code will find the search field element
        try:
            search_field=driver.find_element(By.XPATH,'//textarea[@class="gLFyf"]')
            
        except NoSuchElementException:
            search_field=driver.find_element(By.XPATH,'//input[@class="og3lId"]')
            
        #Code will clear the search field of any previous keywords
        search_field.clear()    
    
        search_field.send_keys(item)
        
        #Code will find the search button and click on it
        try:
            search_button=driver.find_element(By.CSS_SELECTOR, 'button.Tg7LZd')
            driver.execute_script("arguments[0].click();", search_button)
        
        except NoSuchElementException:
            search_button=driver.find_element(By.CSS_SELECTOR, 'div.XZ5MVe')
            driver.execute_script("arguments[0].click();", search_button)
        
        #Scrolling through the executed window for 5 times
        for _ in range(5):
            driver.execute_script("window.scrollBy(0,500)")
            time.sleep(5)
        
        #Code will find all the image elements
        images=driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
        
        #Iterate thorugh all image elements
        for image in images:       
            
            #Scraping the src from image and assigning it to source variable
            source=image.get_attribute('src')
            
            #Applying condition if the source is not null
            if source is not None:
                
                #Applying condition if the 1st 4 string is 'http'
                if source[0:4] =='http':
                    
                    #Applying if condition to check for item equals in keywords
                    if item=='Fruits':
                        fruits_image_source.append(source)
                    
                    elif item=='Cars':
                        cars_image_source.append(source)
    
                    elif item=='Machine Learning':
                        machine_learning_image_source.append(source)
                        
                    elif item=='Guitar':
                        guitar_image_source.append(source)
    
                    elif item=='Cakes':
                        cakes_image_source.append(source)
    
    #Assigning all the lists in image_urls_lists list
    image_urls_lists=[fruits_image_source,cars_image_source,machine_learning_image_source,guitar_image_source,cakes_image_source]        


    #Code to download the images in local machine
    keywords = ['Fruits','Cars','Machine Learning','Guitar','Cakes']
    #Creates a new directory 'Google_images' 
    os.makedirs('Google_images', exist_ok=True)
    
    #Iterates image_urls_list and checks if index=10 break the code
    for i,image_urls_list in enumerate(image_urls_lists):
        for index, url in enumerate(image_urls_list):
            if index==10:
                break
            response=requests.get(url)
            
            #Create image file under Google_images directory with name as 'Fruits_image_0' 
            with open(f"Google_images/{keywords[i]}_image_{index}.jpg", "wb") as file:
                file.write(response.content)

                
Google_Image_Search()

4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [4]:
def Mobile_Keyword():
    
    #Code will take input of the product we wish to search and return the String 
    search_keyword=input("Please enter the Mobile name you wish to search: ")
    return search_keyword


def Flipkart_Smartphone_Search(url):
    
    #Calling the Mobile_Keyword funtion and assigning it to a variable
    keyword=Mobile_Keyword()
    
    driver=webdriver.Chrome()
    driver.get(url)
    
    wait=WebDriverWait(driver,10)
    
    #Creating all the required lists variable
    product_urls = []  
    brand_name = []
    phone_name = []
    phone_color = []
    ram_size = []
    storage_rom = []
    primary_camera = []
    secondary_camera = []
    display_size = []
    battery_capacity = []
    mobile_price = []
    
    #Code to find the search field element
    search = wait.until(EC.visibility_of_element_located((By.XPATH,'//input[@class="Pke_EE"]')))
    search.send_keys(keyword)
    
    #Code to find the search button element and click on it
    search_button = driver.find_element(By.XPATH,'//button[@class="_2iLD__"]')
    search_button.click()
    
    #Code to find all the product elements where url is available
    urls=wait.until(EC.visibility_of_all_elements_located((By.XPATH,'//a[@class="_1fQZEK"]')))
    
    #Iterate through each element to scrap the urls 
    for url in urls:
        product_urls.append(url.get_attribute('href'))
        
    #Iterate through pruduct url    
    for each_product_url in product_urls:
        
        driver.get(each_product_url)
        time.sleep(3)
        
        #These try except blocks of code will scrap all the required data and append it the declared lists
        try:
            name=driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')
            phone_name.append(name.get_attribute("innerText"))
        except NoSuchElementException:
            phone_name.append('-')
        
        try:
            color=driver.find_element(By.XPATH,'//td[@class="_1hKmbr col col-3-12" and contains(text(),"Color")]/following-sibling::td/ul/li')
            phone_color.append(color.get_attribute("innerText"))
        except NoSuchElementException:
            phone_color.append('-')
            
        try:
            ram=driver.find_element(By.XPATH,'//td[@class="_1hKmbr col col-3-12" and contains(text(),"RAM")]/following-sibling::td/ul/li')
            ram_size.append(ram.get_attribute("innerText"))
        except NoSuchElementException:
            ram_size.append('-')
        
        try:
            rom=driver.find_element(By.XPATH,'//td[@class="_1hKmbr col col-3-12" and contains(text(),"Internal Storage")]/following-sibling::td/ul/li')
            storage_rom.append(rom.get_attribute("innerText"))
        except NoSuchElementException:
            storage_rom.append('-')
        
        try:
            p_camera=driver.find_element(By.XPATH,'//td[@class="_1hKmbr col col-3-12" and contains(text(),"Primary Camera")]/following-sibling::td/ul/li')
            primary_camera.append(p_camera.get_attribute("innerText"))
        except NoSuchElementException:
            primary_camera.append('-')
        
        try:
            s_camera=driver.find_element(By.XPATH,'//td[@class="_1hKmbr col col-3-12" and contains(text(),"Secondary Camera")]/following-sibling::td/ul/li')
            secondary_camera.append(s_camera.get_attribute("innerText"))
        except NoSuchElementException:
            secondary_camera.append('-')
        
        try:
            display=driver.find_element(By.XPATH,'//td[@class="_1hKmbr col col-3-12" and contains(text(),"Display Size")]/following-sibling::td/ul/li')
            display_size.append(display.get_attribute("innerText"))
        except NoSuchElementException:
            display_size.append('-')
        
        try:
            battery=driver.find_element(By.XPATH,'//td[@class="_1hKmbr col col-3-12" and contains(text(),"Battery Capacity")]/following-sibling::td/ul/li')
            battery_capacity.append(battery.get_attribute("innerText"))    
        except NoSuchElementException:
            battery_capacity.append('-')
        
        try:
            price=driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')
            mobile_price.append(price.get_attribute("innerText"))
        except NoSuchElementException:
            mobile_price.append('-')
    
    #Creating a DataFrame of the scrapped data
    mobile_df=pd.DataFrame({'Product Name':phone_name,'Color':phone_color,'Ram':ram_size,'Rom':storage_rom,'Primary Camera':primary_camera,'Secondary Camera':secondary_camera,'Display Size':display_size,'Battery capacity':battery_capacity,'Price':mobile_price,'URL':product_urls},index=range(1,len(product_urls)+1))
    
    #Converting df to .csv file
    mobile_df.to_csv('Selenium_WebScraping_Flipkart_phone.csv')
    
    return mobile_df
Flipkart_Smartphone_Search('https://www.flipkart.com/')

Please enter the Mobile name you wish to search: Oneplus Nord


,Product Name,Color,Ram,Rom,Primary Camera,Secondary Camera,Display Size,Battery capacity,Price,URL
1,"OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 256...",Chromatic Gray,8 GB,256 GB,108MP Rear Camera,-,17.07 cm (6.72 inch),5000 mAh,"₹19,925",https://www.flipkart.com/oneplus-nord-ce-3-lit...
2,"OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 128...",Chromatic Gray,8 GB,128 GB,108MP Rear Camera,-,17.07 cm (6.72 inch),5000 mAh,"₹17,528",https://www.flipkart.com/oneplus-nord-ce-3-lit...
3,"OnePlus Nord CE 3 Lite 5G (Pastel Lime, 256 GB...",Pastel Lime,8 GB,256 GB,108MP Rear Camera,-,17.07 cm (6.72 inch),5000 mAh,"₹19,533",https://www.flipkart.com/oneplus-nord-ce-3-lit...
4,"OnePlus Nord CE 3 Lite 5G (Pastel Lime, 128 GB...",Pastel Lime,8 GB,128 GB,108MP Rear Camera,-,17.07 cm (6.72 inch),5000 mAh,"₹17,430",https://www.flipkart.com/oneplus-nord-ce-3-lit...
5,"OnePlus Nord 3 5G (Misty Green, 128 GB) (8 GB...",Misty Green,8 GB,128 GB,50MP Rear Camera,-,17.12 cm (6.74 inch),5000 mAh,"₹26,175",https://www.flipkart.com/oneplus-nord-3-5g-mis...
6,"OnePlus Nord 3 5G (Tempest Gray, 128 GB) (8 G...",Tempest Gray,8 GB,128 GB,Yes,Yes,17.12 cm (6.74 inch),5000 mAh,"₹25,644",https://www.flipkart.com/oneplus-nord-3-5g-tem...
7,"OnePlus Nord CE3 5G (Aqua Surge, 128 GB) (8 G...",Aqua Surge,8 GB,128 GB,50MP Rear Camera,16MP Front Camera,17.02 cm (6.7 inch),5000 mAh,"₹23,640",https://www.flipkart.com/oneplus-nord-ce3-5g-a...
8,"OnePlus Nord CE3 5G (Grey Shimmer, 128 GB) (8...",Grey Shimmer,8 GB,128 GB,50MP Rear Camera,-,17.02 cm (6.7 inch),5000 mAh,"₹23,510",https://www.flipkart.com/oneplus-nord-ce3-5g-g...
9,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...",Black Dusk,6 GB,128 GB,64MP Rear Camera,16MP Front Camera,16.74 cm (6.59 inch),5000 mAh,"₹17,713",https://www.flipkart.com/oneplus-nord-ce-2-lit...
10,"OnePlus Nord CE 2 5G (Gray Mirror, 128 GB) (8...",Gray Mirror,8 GB,128 GB,50MP Rear Camera,-,16.33 cm (6.43 inch),4500 mAh,"₹21,499",https://www.flipkart.com/oneplus-nord-ce-2-5g-...


5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [5]:
def Location():
    
    #Code will take input of the product we wish to search and return the String 
    location=input("Please enter the location you need coordinates for: ")
    return location

def Google_Map_Coordinates(url):
    
    #Calling the Location funtion and assigning it to a variable
    location=Location()
    
    driver=webdriver.Chrome()
    driver.get(url)
    
    wait=WebDriverWait(driver,10)
    
    #Code will find the search field to search location
    location_search = wait.until(EC.visibility_of_element_located((By.XPATH,'//input[@id="searchboxinput"]')))
    
    #Code will send location keyword to the location_search field
    location_search.send_keys(location)
    
    #Click the search button
    driver.find_element(By.XPATH,'//button[@id="searchbox-searchbutton"]').click()
    
    time.sleep(5)
    
    #To grab the current url from the browser
    location_url=driver.current_url
    
    #Using regex to extract the latitude and longitue 
    pattern=r'@(-?\d+\.\d+),(-?\d+\.\d+)'
    location_coordinates=re.search(pattern,location_url)
    
    #Assigning Regex pattern value to latitude and longitude variable
    latitude = float(location_coordinates.group(1))
    longitude = float(location_coordinates.group(2))
    
    #Creating DataFrame 
    df_coordinates=pd.DataFrame({'City':location,'Latitude':latitude,'Longitude':longitude},index=['1'])

    #Returning the DataFrame
    return df_coordinates


Google_Map_Coordinates('https://www.google.com/maps')

Please enter the location you need coordinates for: Bengaluru


,City,Latitude,Longitude
1,Bengaluru,12.95379,77.301265


6. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [6]:
def Best_10_Laptop(url):
    
    driver=webdriver.Chrome()
    driver.get(url)
    
    wait=WebDriverWait(driver,10)
    
    #Creating all the required lists variables
    laptop_name=[]
    operating_system=[]
    display_size=[]
    processors=[]
    
    #Locating the element 'Top 10'
    top_10_tab=wait.until(EC.visibility_of_element_located((By.XPATH,'//li[@id="menu-item-291178"]')))
    top_10_tab.click()
    
    #Locating the element 'Top 10 Best Laptops in India' link
    best_business_laptop_link=wait.until(EC.visibility_of_element_located((By.XPATH,'//p[@class="font90 mobfont80 lineheight20 moblineheight15 mb15"]/a[contains(text(),"Top 10 Best Laptops in India")]')))
    best_business_laptop_link.click()
    
    #Locating all the elements frame of business laptops
    best_business_laptops_frame=wait.until(EC.visibility_of_all_elements_located((By.XPATH,'//div[@class="rh_grid_image_3_col"]')))
    
    #Iterating through each 'best_business_laptops_frame'
    for laptop in best_business_laptops_frame:
        
        #These try except blocks of code will scrap all the required data and append it the declared lists
        try:
            name=laptop.find_element(By.XPATH,'.//div[@class="rh_gr_top_middle mb10 colored_rate_bar"]/h3/a')
            laptop_name.append(name.get_attribute("innerText"))
        except NoSuchElementException:
            laptop_name.append('-')
            
        try:
            os=laptop.find_element(By.XPATH,'.//span[@class="meta_v_label mr5 rtlml5 mr5 rtlml5" and contains(text(),"Operating System")]/following-sibling::span')
            operating_system.append(os.get_attribute("innerText"))
        except NoSuchElementException:
            operating_system.append('-')
        
        try:
            ds=laptop.find_element(By.XPATH,'.//span[@class="meta_v_label mr5 rtlml5 mr5 rtlml5" and contains(text(),"Display Size")]/following-sibling::span')
            display_size.append(ds.get_attribute("innerText"))
        except NoSuchElementException:
            display_size.append('-')
        
        try:
            cpu=laptop.find_element(By.XPATH,'.//span[@class="meta_v_label mr5 rtlml5 mr5 rtlml5" and contains(text(),"Processor")]/following-sibling::span')
            processors.append(cpu.get_attribute("innerText"))
        except NoSuchElementException:
            processors.append('-')
    
    #Creating a DataFrame
    df_best_laptop=pd.DataFrame({'Name':laptop_name,'Operating System':operating_system,'Display Size':display_size,'Processor':processors},index=range(1,len(laptop_name)+1))
    
    #Converting DataFrame to CSV file
    df_best_laptop.to_csv('Best_10_Laptop.csv')
    
    #Returning the DataFrame
    return df_best_laptop

    
Best_10_Laptop('https://www.digit.in/')

,Name,Operating System,Display Size,Processor
1,Apple MacBook Air 2023,Mac,15,-
2,Dell XPS 15,Windows 11 Home,15.6,13th Gen Intel Core i7-13700H
3,Acer Aspire 5 A515-58GM,Windows 11 Home,15.6,13th Gen Intel Core i5-1335U
4,Apple MacBook Pro 16”,MacOS,16,Apple M3 Max
5,ASUS Zenbook 14 OLED (UX3405),Windows 11 Pro,14,Intel Core Ultra 7-155H
6,Acer Predator Helios Neo 16 13th Gen Core i7-1...,Windows 11 Home,16,13th Gen Intel Core i7-13700HX
7,Lenovo Yoga 9i 13th Gen (83B1004XIN),Windows 11 Home,14,13th Gen Intel Core i7-1360P
8,ASUS TUF Gaming A16 Advantage Edition (2023) F...,Windows 11 Home,16,AMD Ryzen 9-7940HS
9,HP Spectre x360 16-f1009TX 12th Gen Core i7-1260P,Windows 11 Home,16,12th Gen Intel EVO Core i7-1260P
10,Samsung Galaxy Book3 Pro NP960XFG-KC2IN,Windows 11 Home,16,13th Gen Intel Core i7-1360P


7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [7]:
def Billionaires_List(url):
    
    driver=webdriver.Chrome()
    driver.get(url)
    
    wait=WebDriverWait(driver,10)
    
    #Locate the dropdown menu in the Forbes 'Home Page' and click on it 
    dropdown_menu=wait.until(EC.visibility_of_element_located((By.XPATH,'//div[@class="_69hVhdY4"]')))
    dropdown_menu.click()
    
    # Initializing ActionChains object for performing mouse actions
    actions=ActionChains(driver)
    
    # Moving mouse to the 'Billionaires' link in the dropdown menu
    billionaire=wait.until(EC.visibility_of_element_located((By.XPATH,'//a[@class="B1S88Gxl" and contains(text(),"Billionaires")]')))
    actions.move_to_element(billionaire).perform()
    
    # Clicking on the 'World's Billionaires' link in the dropdown menu
    world_billionaire=wait.until(EC.visibility_of_element_located((By.XPATH,'''//a[@data-ga-track="U20 - Channel: Billionaires - Position 1 - Section: World's Billionaires"]'''))) 
    actions.move_to_element(world_billionaire).click().perform()
    
    time.sleep(10)
    
    #Creating all the required lists variables
    ranking=[]
    billionaire_names=[]
    net_worth=[]
    billionaire_ages=[]
    citizenship=[]
    source=[]
    industry=[]
    
    #Locating the rank's elements and scrapping the data
    ranks=driver.find_elements(By.XPATH,'//div[@class="Table_rank___YBhk Table_dataCell__2QCve"]')
    for rank in ranks:
        ranking.append(rank.get_attribute("innerText"))
    
    #Scraping the 1st row data in the Billionaire's list
    name = driver.find_element(By.XPATH,'//div[@class="TableRow_row__L-0Km TableRow_activeRow__g4oSF"]/div[2]')
    billionaire_names.append(name.get_attribute("innerText"))
    
    net = driver.find_element(By.XPATH,'//div[@class="TableRow_row__L-0Km TableRow_activeRow__g4oSF"]/div[3]')
    net_worth.append(net.get_attribute("innerText"))
    
    age = driver.find_element(By.XPATH,'//div[@class="TableRow_row__L-0Km TableRow_activeRow__g4oSF"]/div[4]')
    billionaire_ages.append(age.get_attribute("innerText"))
    
    country = driver.find_element(By.XPATH,'//div[@class="TableRow_row__L-0Km TableRow_activeRow__g4oSF"]/div[5]')
    citizenship.append(country.get_attribute("innerText"))
    
    scr = driver.find_element(By.XPATH,'//div[@class="TableRow_row__L-0Km TableRow_activeRow__g4oSF"]/div[6]')
    source.append(scr.get_attribute("innerText"))
    
    inc = driver.find_element(By.XPATH,'//div[@class="TableRow_row__L-0Km TableRow_activeRow__g4oSF"]/div[7]')
    industry.append(inc.get_attribute("innerText"))    
    
    
    #Extracting the names of billionaires starting from the 2nd row
    names = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[2]')
    for name in names:
        billionaire_names.append(name.get_attribute("innerText"))
    
    #Extracting the net worth of billionaires starting from the 2nd row
    nets= driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[3]')   
    for net in nets:
        net_worth.append(net.get_attribute("innerText"))
        
    #Extracting the ages of billionaires starting from the 2nd row
    ages=driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[4]')
    for age in ages:
        billionaire_ages.append(age.get_attribute("innerText"))
        
    #Extracting the countires of billionaires starting from the 2nd row
    countries = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[5]')
    for country in countries:
        citizenship.append(country.get_attribute("innerText"))
    
    #Extracting the source of billionaires starting from the 2nd row
    sources = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[6]')
    for scr in sources:
        source.append(scr.get_attribute("innerText"))
        
    #Extracting the industries of billionaires starting from the 2nd row
    industries = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[7]')
    for inc in industries:
        industry.append(inc.get_attribute("innerText"))
    
    #Creating a DataFrame
    df_billionaires=pd.DataFrame({'Rankings':ranking,'Name':billionaire_names,'Net Worth':net_worth,'Age':billionaire_ages,'Country':citizenship,'Source':source,'Industry':industry},index=range(1,len(ranking)+1))
    
    #Return the DataFrame
    return df_billionaires


Billionaires_List('https://www.forbes.com/')

,Rankings,Name,Net Worth,Age,Country,Source,Industry
1,1,Bernard Arnault & family,$211 B,74,France,LVMH,Fashion & Retail
2,2,Elon Musk,$180 B,51,United States,"Tesla, SpaceX",Automotive
3,3,Jeff Bezos,$114 B,59,United States,Amazon,Technology
4,4,Larry Ellison,$107 B,78,United States,Oracle,Technology
5,5,Warren Buffett,$106 B,92,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
196,195,Jin Baofang,$9.6 B,70,China,Solar panels,Energy
197,195,Luo Liguo & family,$9.6 B,67,China,Chemicals,Manufacturing
198,195,Marijke Mars,$9.6 B,58,United States,"Candy, pet food",Food & Beverage
199,195,Pamela Mars,$9.6 B,62,United States,"Candy, pet food",Food & Beverage


8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [8]:
def YouTube_Comments(url):
    
    driver=webdriver.Chrome()
    driver.get(url)
    
    wait = WebDriverWait(driver,10)
    
    #Creating all the required lists variables
    all_comments = []
    likes = []
    time_of_posting = []
    
    #Scrolling through the executed window
    for _ in range(500):
        driver.execute_script("window.scrollBy(0,500)")
        time.sleep(3)
    
    #Locating all the comment frames of the youtube link
    youtube_comment_frame=driver.find_elements(By.XPATH,'//div[@id="main"]')
    
    #Iterating through each youtube comment frame
    for each_frame in youtube_comment_frame:
        #Extracting the comment and appending it to all_comments
        comment=each_frame.find_element(By.XPATH,'.//div[@class="style-scope ytd-expander"]')
        all_comments.append(comment.text)
        
        #Extracting the no. of likes and appending it to likes
        likes_count=each_frame.find_element(By.XPATH,'.//span[@id="vote-count-middle"]')
        likes.append(likes_count.text)
        
        #Extracting the time of posting and appending it to time_of_posting
        posted_time = each_frame.find_element(By.XPATH,'.//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]')
        time_of_posting.append(posted_time.text)   
        
    
    #Creating a DataFrame
    df_youtube_comments=pd.DataFrame({'Comments':all_comments,'Total Likes':likes,'Time of posting':time_of_posting},index=range(1,len(all_comments)+1))    
    
    #Returning the DataFrame
    return df_youtube_comments
    
    
YouTube_Comments('https://www.youtube.com/watch?v=8MwxeYnyrRs')

,Comments,Total Likes,Time of posting
1,"Hello, to start with, I really like your inter...",978,3 years ago
2,Какие красивые и мужественные лица у этого гор...,24,4 months ago
3,"Here in mid-2023, listen to this rarity!!! Who...",58,11 months ago
4,Замечательная музыка проникают до самого серд...,87,1 year ago
5,"This music is magical, like bringing memories ...",23,10 months ago
...,...,...,...
1230,"Истребили целую нацию, забрали землю и играют ...",,8 months ago
1231,As lições e ensinamentos dos indígenas america...,,1 year ago
1232,아메리카 원주민들은 동양인.그것도 동북아. 고조선의 유민들이 아닌가?!,,1 year ago
1233,Аграрии США в УКРАИНЕ выращивают ГМО. Дёшево ...,,1 month ago


9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [9]:
def HostelWorld_Data(url):
    
    driver=webdriver.Chrome()
    driver.get(url)
    
    wait=WebDriverWait(driver,10)
    
    #Locating the location element and send 'London' keyword 
    location=wait.until(EC.visibility_of_element_located((By.XPATH,'//label[@class="input-label" and contains(text()," Where do you want to go? ")]/preceding-sibling::input')))
    location.send_keys('London')
    
    # Code to click the drop down list when appears after entering the 'London' keyword in loaction field 
    dropdown_location=wait.until(EC.visibility_of_element_located((By.XPATH,'//li[@class="select-item-group" and contains(text(),"City")]/following-sibling::li[@type="city"]')))
    dropdown_location.click()
    
    #Code to locate and click the 'Let's Go'button
    letsgo_button=driver.find_element(By.XPATH,'''//span[@class="text" and contains(text(),"Let's go!")]''')
    letsgo_button.click()
    
    time.sleep(10)
    
    #Creating all the required lists variables
    hostel_name = []
    hostel_urls = []
    distance_from_citycenter = []
    hostel_rating = []
    total_reviews = []
    overall_reviews = []
    privates_from_price = []
    dorms_from_price = []
    facilities_provided = []
    property_description = []

    #Conditional state to iterate through all the pages
    while True:
        
        #Locating all the property elements where all the required elements are available
        property_frame=driver.find_elements(By.XPATH,'//div[@class="property-card"]')
        
        #Iterating through each property_frame
        for each_property in property_frame:
            
            try:
                #Code to extract only the property_frames where the 'city_center' element is available, else continue
                city_center=each_property.find_element(By.XPATH,'.//span[@class="distance-description"]')
                distance_from_citycenter.append(city_center.get_attribute("innerText"))
                
                #Code to extract the url of each hostel and append it in list
                try:
                    url=each_property.find_element(By.XPATH,'.//a[@rel="noreferrer noopener"]')
                    hostel_urls.append(url.get_attribute('href'))
                except NoSuchElementException:
                    hostel_urls.append('-')
                
                #Code to extract the name of each hostel and append it in list
                try:
                    name=each_property.find_element(By.XPATH,'.//div[@class="property-name"]/span')
                    hostel_name.append(name.get_attribute("innerText"))
                except NoSuchElementException:
                    hostel_name.append('-')
                
                #Code to extract the rating of each hostel and append it in list
                try:
                    rating=each_property.find_element(By.XPATH,'.//span[@class="number"]')
                    hostel_rating.append(rating.get_attribute("innerText"))
                except NoSuchElementException:
                    hostel_rating.append('-')
                
                #Code to extract the no. of reviews of each hostel and append it in list
                try:    
                    no_of_review=each_property.find_element(By.XPATH,'.//div[@class="review"]/span')
                    total_reviews.append(no_of_review.get_attribute("innerText"))
                except NoSuchElementException:
                    total_reviews.append('-')
                
                #Code to extract the overall reviews of each hostel and append it in list
                try:
                    review=each_property.find_element(By.XPATH,'.//span[@class="keyword"]')
                    overall_reviews.append(review.get_attribute("innerText"))
                except NoSuchElementException:
                    overall_reviews.append('-')
                
                #Code to extract the 'private from' of each hostel and append it in list
                try:
                    private_from=each_property.find_element(By.XPATH,'.//div[@class="accommodation-label" and contains(text(),"Privates From")]/following-sibling::div/strong')
                    privates_from_price.append(private_from.get_attribute("innerText"))
                except NoSuchElementException:
                    privates_from_price.append('-')
                
                #Code to extract the 'dorm from' of each hostel and append it in list
                try:
                    dorm_from=each_property.find_element(By.XPATH,'.//div[@class="accommodation-label" and contains(text(),"Dorms From")]/following-sibling::div/strong')
                    dorms_from_price.append(dorm_from.get_attribute("innerText"))
                except NoSuchElementException:
                    dorms_from_price.append('-')
                
        
            except NoSuchElementException:
                continue
        
        #Code to locate the next button and click on it       
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.XPATH,'//button[@class="pill-content page-nav nav-right icon-only"]')))
            driver.execute_script("arguments[0].click();", next_button)
            
        except (NoSuchElementException,TimeoutException):
            break
    
    #Iterate through each hostel urls 
    for url in hostel_urls:
        driver.get(url)
        time.sleep(5)
        
        #Code to extract property description and append it
        try:
            desc=driver.find_element(By.XPATH,'//div[@class="description-container"]/div/div[2]')
            property_description.append(desc.get_attribute("innerText"))
        except NoSuchElementException:
            property_description.append('-')
        
        #Code to extract facilities 
        facilities_elements=driver.find_elements(By.XPATH,'//ul[@class="facilities"]')
        for facilities_list in facilities_elements:
            all_facilities = facilities_list.find_elements(By.XPATH,'//ul[@class="facilities"]/li[@data-v-5642c271]')
            facility_string = ', '.join([facility.text.strip() for facility in all_facilities])
        
        #Code to append all the strings in a  list   
        facilities_provided.append(facility_string)

    #Creating a DataFrame
    df_hostelworld_london=pd.DataFrame({'Hostel name':hostel_name,'Distance from City center':distance_from_citycenter,'Ratings':hostel_rating,'Total reviews':total_reviews,'Overall Reviews':overall_reviews,'Private from price':privates_from_price,'Dorm from price':dorms_from_price,'Property Description':property_description,'Facilities':facilities_provided,'URL':hostel_urls},index=range(1,len(hostel_name)+1))
    
    #Return the DataFrame
    return df_hostelworld_london
    
    
HostelWorld_Data('https://www.hostelworld.com/')

,Hostel name,Distance from City center,Ratings,Total reviews,Overall Reviews,Private from price,Dorm from price,Property Description,Facilities,URL
1,Wombat's City Hostel London,- 3.6km from city centre,9.2,(16140),Superb,"₹23,788","₹3,649","\nWombat's City Hostel London, located close t...","Linen Included, Free City Maps, Free WiFi, Fre...",https://www.hostelworld.com/pwa/hosteldetails....
2,Palmers Lodge - Swiss Cottage,- 6.5km from city centre,8.7,(16051),Fabulous,-,"₹2,104",\nPalmers Lodge Swiss Cottage is set in what o...,"Linen Included, Free WiFi, Free Internet Acces...",https://www.hostelworld.com/pwa/hosteldetails....
3,NX London Hostel,- 6.1km from city centre,8.9,(2252),Fabulous,"₹10,515","₹2,545",\nWelcome to NX London Hostel!\n\nPLEASE READ ...,"Free Breakfast, Linen Included, Towels Include...",https://www.hostelworld.com/pwa/hosteldetails....
4,Urbany Hostel London,- 5.4km from city centre,9.6,(1186),Superb,"₹10,516","₹2,675",\nWelcome to Urbany Hostel London! Ideal for t...,"Linen Included, Free WiFi, Security Lockers, M...",https://www.hostelworld.com/pwa/hosteldetails....
5,Onefam Notting Hill,- 5.5km from city centre,9.6,(2704),Superb,"₹15,298","₹2,647",\nThe perfect place for solo travelers to conn...,"Linen Included, Free WiFi, Free Internet Acces...",https://www.hostelworld.com/pwa/hosteldetails....
...,...,...,...,...,...,...,...,...,...,...
64,Europa House Hotel,- 4.2km from city centre,3.7,(1),-,"₹7,159",-,"\nWe are a mid-terrace house, built in approxi...","Linen Included, Free City Maps, Towels Include...",https://www.hostelworld.com/pwa/hosteldetails....
65,Hyde Park Court Hotel,- 4.3km from city centre,5.0,(139),-,"₹10,002",-,\nThe Hyde Park Court Hotel is a centrally loc...,"Free Breakfast, Free City Maps, Free WiFi, Fre...",https://www.hostelworld.com/pwa/hosteldetails....
66,Cranbrook Hotel,- 14.8km from city centre,-,-,-,"₹5,545",-,\nWe are located about twenty minutes by tube ...,"Free Parking, Hot Showers, Parking, Safe Depos...",https://www.hostelworld.com/pwa/hosteldetails....
67,Aron Guest House,- 13.1km from city centre,2.0,(27),-,"₹8,291",-,"\nAron Guest House provides an excellent base,...","Free Parking, Towels Included, Free WiFi, Park...",https://www.hostelworld.com/pwa/hosteldetails....
